We are solving the following linear program using Simplex Method:

$$\text{max}\;\;3x_1 + x_2 + 2x_3$$
$$\text{s.t.}\;\;x_1 + x_2 + 3x_3 \leq 30$$
$$2x_1 + 2x_2 + 5x_3 \leq 24$$
$$4x_1 + x_2 + 2x_3 \leq 36$$
$$x_1,\;x_2,\;x_3 \geq 0$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import HalfspaceIntersection
import math

In [20]:
# Plotting the inequalities and identifying the feasible region by discarding the infeasible region

# halfspaces = np.array([
#     [-1, 0, 0, 0],    # x1 ≥ 0
#     [0, -1, 0, 0],    # x2 ≥ 0
#     [0, 0, -1, 0],    # x3 ≥ 0
#     [1, 1, 3, -30],   # x1 + x2 + 3x3 ≤ 30
#     [2, 2, 5, -24],   # 2x₁ + 2x2 + 5x3 ≤ 24
#     [4, 1, 2, -36],   # 4x1 + x2 + 2x3 ≤ 36
# ])
# feasible_point = np.array([0.5, 0.5, 0.5])
# hs = HalfspaceIntersection(halfspaces, feasible_point)

# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1, aspect='equal')
# xlim, ylim = (0, 10), (0, 10)
# ax.set_xlim(xlim)
# ax.set_ylim(ylim)
# x = np.linspace(0, 10, 100)

# signs = [-1, -1, -1, 0, 0, 0]
# for h, sign in zip(halfspaces, signs):
#     hlist = h.tolist()
#     if h[1]== 0:
#         ax.axvline(-h[2]/h[0], label='{}x+{}y+{}z+{}=0'.format(*hlist))
#         xi = np.linspace(xlim[sign], -h[2]/h[0], 100)
#         ax.fill_between(xi, ylim[0], ylim[1], color='grey', alpha=0.5)
#     else:
#         ax.plot(x, (-h[2]-h[0]*x)/h[1], label='{}x+{}y+{}z+{}=0'.format(*hlist))
#         ax.fill_between(x, (-h[2]-h[0]*x)/h[1], ylim[sign], color='grey', alpha=0.5)
# x, y, z = zip(*hs.intersections)
# ax.plot(x, y, 'o', markersize=5)
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)

In [10]:
# Coefficient matrix of the objective function
c = [3, 1, 2, 0, 0, 0]

# Coefficients of the inrquality constraints including the slack variables
A = [
    [1, 1, 3, 1, 0, 0],
    [2, 2, 5, 0, 1, 0],
    [4, 1, 2, 0, 0, 1]
]

# Right hand sides of the inequality constraints
b = [30, 24, 36]

def to_tableau(c, A, b):
    '''
    Create the tableau from the coefficient matrices and return the tableau as a matrix with the last row corresponding to the
    objective function.
    '''
    xb = [eq + [x] for eq, x in zip(A, b)] # matrix with rows corresponding to the constraints
    z = c + [0] # row corresponding to the objective function
    return xb + [z]

def can_be_improved(tableau):
    '''
    Checks if the current objective value can be improved further by checking if there is any +ve value remaining in the row
    (last row) of the tableau corresponding to the objective function. If there is any, then the objective value can be improved,
    and the function returns true. Otherwise, it returns false.
    '''
    z = tableau[-1]
    return any(x > 0 for x in z[:-1]) # checks if there is any +ve velue the row corresponding to the objective function

def get_pivot_position(tableau):
    '''
    Identify the pivot position from the current tableau and returns the corresponding row and column. It is the intersection of the
    largest positive element in the objective function row (defines the pivot column) and smallest non-negative ratio (defines the 
    pivot row) which is the ratio between LHS (values from the inequalities in the pivot column) and the corresponding b value.
    '''
    # IDENTIFY THE PIVOT COLUMN
    z = tableau[-1] # extracts the last row of the tableau, which typically represents the objective function row
    # identifies the entering variable by finding the first column (ignoring the last element which is the objective function value)
    # with a positive coefficient.
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    # print(column, z[column])
    '''
    **IMPROVEMENT**: Select the maximum element from the objective row as the pivot column. Right now, it selects the first positive
    element.
    '''

    # IDENTIFY THE PIVOT ROW
    restrictions = [] # empty list to store the ratios for the minimum ratio test 
    for eq in tableau[:-1]: # selects every row corresponding to the inequalities except the last row (objective function)
        el = eq[column] # take the LHS for each row (the element in the column of the entering variable)
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)
        # apply minimum ratio test: divide the b value with the LHS to find all the raios. if the ratio is -ve, discard it by
        # assigning infinity value

    row = restrictions.index(min(restrictions)) # we select the row with the minimum non-negative ratio
    return row, column

def pivot_step(tableau, pivot_position):
    '''
    Creates the new tableau from the existing tableau. It takes two parameters, the old tableau and the pivot_position (row and
    column of the pivot element) and it returns the new tableau.
    '''
    new_tableau = [[] for eq in tableau]
    # creates an empty list for each row in the original tableau to hold the rows of the new tableau after the pivot operation.
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value # update pivot row
    # normalizes the pivot row by dividing each element by the pivot value to make the pivot element equal to 1 in the new tableau.
    
    for eq_i, eq in enumerate(tableau): # iterate over each row in the original tableau
        if eq_i != i: # skip the pivot row since it is already updated
            # compute the multipler to zero out the element in the pivot column of the current row
            # multiplier = normalized pivot row * element in the current row's pivot column
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
            # subtract the multiplier from the current row to update it, ensuring that element in the pivot column becomes zero.
   
    return new_tableau

def is_basic(column):
    '''
    Checks if the column (passed as an argument) is a basic column and returns True or False.
    Basic columns should have only one element equal to 1 and rest of the elements should be 0. This can be validated by checking
    whether the sum of all elements equals to 1 and whether the count of 0 elements is one less than the number of total elements.
    '''
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    '''
    Extract the solution vector from the final Simplex tableau.
    '''
    columns = np.array(tableau).T # take the transpose (makes it easier to iterate through each column of the original tableau)
    # initialize an empty list to store the solutions. If it is a basic variable, then the value will get updated, otherwise not.
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            # if the column is part of the feasible basic solution, convert the column to a list and find the position of the 1.
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index] # extract the solution (corresponding value of the RHS column of the tableau)
        solutions.append(solution)
        
    return solutions

def simplex(c, A, b):
    '''
    Capsule function which sequentially calls the other functions for simplex algorithm working
    '''
    tableau = to_tableau(c, A, b)

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

solution = simplex(c, A, b)
total_sum = sum(a * b for a, b in zip(solution, c))
'''
**IMPROVEMENT**: Print the set of basic variables and non-basic variables separately
'''
print('Optimal Solution: ', solution)
print('Maximized Objective Value: ', total_sum)

Optimal Solution:  [8.0, 4.0, 0, 18.0, 0, 0, 0]
Maximized Objective Value:  28.0
